In [1]:
# Импортируем необходимые библиотеки и функции
from dash import Dash, html, dcc, callback, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import math

In [2]:
# Читаем данные из файла
df = pd.read_csv('games.csv')

In [3]:
df.head()

,Name,Platform,Year_of_Release,Genre,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,NaN,NaN,NaN


In [4]:
# Оставляем данные с 1990 по 2010 год
df = df[(df['Year_of_Release'] >= 1990) & (df['Year_of_Release'] <= 2010)]

In [5]:
# Удаляем строки с пропусками
df = df.dropna()

In [6]:
# Удаляем строки-дубликаты, если они есть
df.drop_duplicates(inplace=True)

In [7]:
# Посмотрим на типы данных
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6124 entries, 0 to 16702
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             6124 non-null   object 
 1   Platform         6124 non-null   object 
 2   Year_of_Release  6124 non-null   float64
 3   Genre            6124 non-null   object 
 4   Critic_Score     6124 non-null   float64
 5   User_Score       6124 non-null   object 
 6   Rating           6124 non-null   object 
dtypes: float64(2), object(5)
memory usage: 382.8+ KB


In [8]:
df.User_Score.unique()

array(['8', '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4', '9',
       '7.9', '8.7', '7.1', '8.9', '6.4', '7.8', '7.5', '9.2', '7.3',
       '8.2', '7.6', '9.1', '8.8', '8.1', '9.4', '6.8', '7.2', '6.5',
       '5.4', '9.3', '6', 'tbd', '4', '6.9', '6.7', '4.6', '7', '5.3',
       '6.1', '5.7', '4.3', '6.2', '5.5', '5.2', '5.6', '5.9', '3.3',
       '4.1', '4.4', '4.5', '1.9', '3.1', '5', '5.8', '2', '9.5', '5.1',
       '3.4', '4.7', '2.6', '2.1', '3.6', '4.8', '4.2', '4.9', '3', '2.9',
       '9.6', '3.7', '3.9', '2.8', '1.7', '3.5', '2.7', '3.8', '2.4',
       '3.2', '1.2', '2.5', '2.3', '0.5', '1.8', '0.6', '0.9', '1', '1.4',
       '1.5', '0.7', '2.2'], dtype=object)

In [9]:
1 - (df.query('User_Score != "tbd"').shape[0] / df.shape[0])

0.16476159372958854

Согласно задаче из данных нужно исключить проекты, для которых имеются пропуски данных в любой из колонок.<br>
'tbd' - это не явное пустое значение, то есть это не NaN, но `User_Score` в данном случае нам неизвестен.<br>
Это значит, что игра еще не получила достаточного количества оценок игроков для расчета `User_Score`, то есть `User_Score` у таких игр сейчас невозможно определить.<br>
Доля проектов с 'tbd' в `User_Score` ≈ 16% среди всех проектов 1990-2010 гг. без пропусков данных в любой из колонок, то есть мы не должны сильно потерять, если уберем их из датафрейма.<br>
Итого будем использовать только те проекты, по которым известны все переменные: `Name`, `Platform`, `Year_of_Release`, `Genre`, `Critic_Score`, `User_Score` и `Rating`.

In [10]:
df = df.query('User_Score != "tbd"')

In [11]:
df = df.astype({'Year_of_Release': 'int', 'User_Score': 'float'})

In [12]:
df.Rating.unique()

array(['E', 'M', 'T', 'E10+', 'AO', 'K-A'], dtype=object)

Будем использовать информацию о рейтинге со страницы https://www.esrb.org/ratings-guide.

In [13]:
# Преобразование рейтинга в числовой вид
rating_map = {
    'E': 0,  # Everyone
    'K-A': 0, # Kids to Adults (was used until the year 1998 when it renamed to E)
    'E10+': 10,  # Everyone 10 and older
    'T': 13,  # Teen
    'M': 17,  # Mature
    'AO': 18  # Adults Only
}

In [14]:
# Присваиваем числовые значения строковым и создаем новый столбец
df['Numeric_Rating'] = df['Rating'].map(rating_map)

In [15]:
# Создание приложения Dash
app = Dash(__name__)

# Макет дашборда
app.layout = html.Div([
    html.Div([
        html.H1('Game Industry Dashboard', style={'fontFamily': '"Open Sans", verdana, arial, sans-serif', 'font-size': '36px'}),
        html.P([
            'This dashboard analyzes video game trends from 1990 to 2010. The dashboard is based on games for which all the following data is known: name, platform, release year, genre, user and critic ratings and age ratings.', html.Br(),
            'Use the filters to explore game releases by platform, genre and release year. Key metrics like total games, average user and critic ratings and age ratings are displayed, with interactive charts updating based on your selections'
        ], style={'fontFamily': '"Open Sans", verdana, arial, sans-serif', 'font-size': '18px'}),
    ], style={'backgroundColor': 'white', 'padding': '20px', 'margin-bottom': '20px'}),
    
    # Фильтры
    html.Div([
        html.Div([
            html.Div([
                html.Label('Platform', style={'fontFamily': '"Open Sans", verdana, arial, sans-serif', 'font-size': '22px'}),
                dcc.Dropdown(
                    id='platform_filter',
                    options=[{'label': i, 'value': i} for i in sorted(df['Platform'].unique())],
                    value=df['Platform'].unique(),
                    multi=True,
                    style={'fontFamily': '"Open Sans", verdana, arial, sans-serif', 'font-size': '18px'}
                )
            ], style={'display': 'inline-block', 'width': '33%', 'padding-right': '10px', 'vertical-align': 'top', 'textAlign': 'center'}),
        
            html.Div([
                html.Label('Genre', style={'fontFamily': '"Open Sans", verdana, arial, sans-serif', 'font-size': '22px'}),
                dcc.Dropdown(
                    id='genre_filter',
                    options=[{'label': i, 'value': i} for i in sorted(df['Genre'].unique())],
                    value=df['Genre'].unique(),
                    multi=True,
                    style={'fontFamily': '"Open Sans", verdana, arial, sans-serif', 'font-size': '18px'}
                )
            ], style={'display': 'inline-block', 'width': '33%', 'padding-right': '10px', 'vertical-align': 'top', 'textAlign': 'center'}),

            html.Div([
                html.Label('Release Year', style={'fontFamily': '"Open Sans", verdana, arial, sans-serif', 'font-size': '22px'}),
                dcc.RangeSlider(
                    id='year_filter',
                    min=df['Year_of_Release'].min(),
                    max=df['Year_of_Release'].max(),
                    step=1,
                    value=[1990, 2010],
                    marks={year: {'label': str(year), 
                                  'style': {'fontFamily': '"Open Sans", verdana, arial, sans-serif', 'font-size': '18px'}} for year in range(1990, 2011, 2)}
                )
            ], style={'display': 'inline-block', 'width': '33%', 'vertical-align': 'top', 'textAlign': 'center'}),
        ], style={'display': 'flex', 'margin-bottom': '20px'}),
    ], style={'backgroundColor': 'white', 'padding': '10px', 'margin-bottom': '20px'}),
    
    # Графики
    html.Div([
        # График 1: Общее число игр
        html.Div(dcc.Graph(id='total_games'), style={'display': 'inline-block', 'width': '33%'}),
        # График 2: Средняя оценка игроков
        html.Div(dcc.Graph(id='avg_user_score'), style={'display': 'inline-block', 'width': '33%'}),
        # График 3: Средняя оценка критиков
        html.Div(dcc.Graph(id='avg_critic_score'), style={'display': 'inline-block', 'width': '33%'}),
    ], style={'display': 'flex', 'justify-content': 'space-between', 'margin-bottom': '20px'}),

    html.Div([
        # График 4: Средний возрастной рейтинг по жанрам
        html.Div(dcc.Graph(id='avg_age_by_genre'), style={'display': 'inline-block', 'width': '33%'}),
        # График 5: Оценки игроков и критиков
        html.Div(dcc.Graph(id='user_vs_critic_score_by_genre'), style={'display': 'inline-block', 'width': '33%'}),
        # График 6: Выпуск игр по годам и платформам
        html.Div(dcc.Graph(id='release_by_year_and_platform'), style={'display': 'inline-block', 'width': '33%'}),
    ], style={'display': 'flex', 'justify-content': 'space-between'})
], style={'backgroundColor': '#F0F0F0',
          'padding-top': '20px', 'padding-left': '20px', 'padding-right': '20px', 'padding-bottom': '20px'})

In [16]:
# Обновление графиков
@callback(
    [Output('total_games', 'figure'),
     Output('avg_user_score', 'figure'),
     Output('avg_critic_score', 'figure'),
     Output('avg_age_by_genre', 'figure'),
     Output('user_vs_critic_score_by_genre', 'figure'),
     Output('release_by_year_and_platform', 'figure')],
    [Input('platform_filter', 'value'),
     Input('genre_filter', 'value'),
     Input('year_filter', 'value')]
)
def update_graphs(selected_platforms, selected_genres, selected_years):
    # Фильтрация данных
    filtered_df = df[(df['Platform'].isin(selected_platforms)) &
                     (df['Genre'].isin(selected_genres)) &
                     (df['Year_of_Release'] >= selected_years[0]) &
                     (df['Year_of_Release'] <= selected_years[1])]

    # График 1: Общее число игр
    total_games_count = filtered_df.shape[0]
    fig1 = go.Figure(go.Indicator(
        mode='number',
        value=total_games_count,
        title={'text': 'Total Games'}
    ))
    fig1.update_layout(height=250)

    # График 2: Средняя оценка игроков
    avg_user_score = filtered_df['User_Score'].mean()
    fig2 = go.Figure(go.Indicator(
        mode='number',
        value=avg_user_score,
        title={'text': 'Average User Score'}
    ))
    fig2.update_layout(height=250)

    # График 3: Средняя оценка критиков
    avg_critic_score = filtered_df['Critic_Score'].mean()
    fig3 = go.Figure(go.Indicator(
        mode='number',
        value=avg_critic_score,
        title={'text': 'Average Critic Score'}
    ))
    fig3.update_layout(height=250)

    # График 4: Средний возрастной рейтинг по жанрам
    avg_age_by_genre = filtered_df.groupby('Genre')['Numeric_Rating'].mean().reset_index()
    avg_age_by_genre['Numeric_Rating'] = avg_age_by_genre['Numeric_Rating'].apply(lambda x: math.floor(x)) # Округление в меньшую сторону
    fig4 = px.bar(avg_age_by_genre, x='Genre', y='Numeric_Rating', title='Average Age Rating by Genre',
                  labels={'Numeric_Rating': 'Average Age Rating'}) # Меняем название оси Y
    fig4.update_traces(text=[f'{int(value)}+' for value in avg_age_by_genre['Numeric_Rating']], # Лейбл с +
                       textposition='outside') # Лейбл над столбцами
    fig4.update_layout(
        title_x=0.5, # Центрируем название
        hovermode=False, # Убираем hover
        yaxis=dict(
            range=[0, avg_age_by_genre['Numeric_Rating'].max() + 2], # Добавляем запас сверху
            tickvals=list(range(0, int(avg_age_by_genre['Numeric_Rating'].max()) + 2, 2)), # Шаг оси Y
            ticktext=[f'{x}+' for x in range(0, int(avg_age_by_genre['Numeric_Rating'].max()) + 2, 2)] # Добавляем + на оси Y
        ),
        title_font=dict(size=20) # Меняем размер шрифта
    )

    # График 5: Оценки игроков и критиков
    fig5 = px.scatter(filtered_df, x='Critic_Score', y='User_Score', color='Genre',
                      title='User and Critic Score by Genre',
                      labels={'User_Score': 'User Score', 'Critic_Score': 'Critic Score'}) # Меняем названия осей X и Y
    fig5.update_layout(title_x=0.5, title_font=dict(size=20)) # Центрируем название и меняем размер шрифта

    # График 6: Выпуск игр по годам и платформам
    release_by_year_and_platform = filtered_df.groupby(['Year_of_Release', 'Platform']).size().reset_index(name='Number of Games')
    fig6 = px.area(release_by_year_and_platform, x='Year_of_Release', y='Number of Games', color='Platform',
                   title='Release by Year and Platform',
                   labels={'Year_of_Release': 'Release Year'}) # Меняем название оси X
    fig6.update_layout(title_x=0.5, title_font=dict(size=20)) # Центрируем название и меняем размер шрифта
    fig6.update_xaxes(dtick=1) # Шаг в 1 год

    return fig1, fig2, fig3, fig4, fig5, fig6

In [17]:
# Запуск приложения
if __name__ == '__main__':
    app.run(debug=True, jupyter_mode='tab')

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>